# Multi-Phase Forecasting

This notebook demonstrates simultaneous forecasting of oil, gas, and water production while maintaining physical relationships.

## Learning Objectives

By the end of this notebook, you will be able to:
- Create multi-phase data structures
- Forecast oil, gas, and water simultaneously
- Maintain physical relationships (GOR, water cut)
- Evaluate multi-phase forecast quality

## Table of Contents
1. [Setup](#setup)
2. [Multi-Phase Data Structure](#data)
3. [Coupled Forecasting](#coupled)
4. [Physical Relationships](#relationships)
5. [Evaluation](#evaluation)

## 1. Setup {#setup}

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from decline_curve.multiphase import MultiPhaseData, MultiPhaseForecaster
from decline_curve.plot import minimal_style

minimal_style()
%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 8)

print("✓ All imports successful!")

## 2. Multi-Phase Data Structure {#data}

In [ ]:
# Generate realistic multi-phase production data
np.random.seed(42)
dates = pd.date_range('2020-01-01', periods=36, freq='MS')

# Oil production (declining)
oil = pd.Series(1000 * np.exp(-0.01 * np.arange(36)), index=dates)

# Gas production (declining GOR)
gor_initial = 1500  # SCF/STB
gor = gor_initial * np.exp(-0.005 * np.arange(36))
gas = oil * gor / 1000  # Convert to MCF

# Water production (increasing water cut)
water_cut_initial = 0.05  # 5%
water_cut = water_cut_initial + 0.002 * np.arange(36)
water = oil * water_cut / (1 - water_cut)

# Create MultiPhaseData object
multiphase_data = MultiPhaseData(oil=oil, gas=gas, water=water)

print("=" * 70)
print("MULTI-PHASE DATA SUMMARY")
print("=" * 70)
print(f"Period: {dates[0].strftime('%Y-%m')} to {dates[-1].strftime('%Y-%m')}")
print(f"Data points: {len(multiphase_data.dates)}")
print(f"\nOil: {multiphase_data.oil.sum():,.0f} bbl cumulative")
print(f"Gas: {multiphase_data.gas.sum():,.0f} MCF cumulative")
print(f"Water: {multiphase_data.water.sum():,.0f} bbl cumulative")
print("=" * 70)

## 3. Coupled Forecasting {#coupled}

In [ ]:
# Create forecaster
forecaster = MultiPhaseForecaster()

# Forecast with enforced ratios (maintains GOR and water cut)
forecast = forecaster.forecast(
    multiphase_data,
    horizon=24,
    enforce_ratios=True
)

print("=" * 70)
print("MULTI-PHASE FORECAST")
print("=" * 70)
print(f"Forecast horizon: 24 months")
print(f"\nOil forecast range: {forecast['oil'].min():.1f} - {forecast['oil'].max():.1f} bbl/month")
print(f"Gas forecast range: {forecast['gas'].min():.1f} - {forecast['gas'].max():.1f} MCF/month")
print(f"Water forecast range: {forecast['water'].min():.1f} - {forecast['water'].max():.1f} bbl/month")
print("=" * 70)

## 4. Physical Relationships {#relationships}

In [ ]:
# Calculate ratios for historical and forecast
historical_ratios = multiphase_data.calculate_ratios()

# Create forecast MultiPhaseData to calculate ratios
forecast_data = MultiPhaseData(
    oil=forecast['oil'],
    gas=forecast['gas'],
    water=forecast['water']
)
forecast_ratios = forecast_data.calculate_ratios()

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Oil production
axes[0, 0].plot(multiphase_data.dates, multiphase_data.oil.values, 'o-', label='Historical', linewidth=2)
axes[0, 0].plot(forecast['oil'].index, forecast['oil'].values, '-', label='Forecast', linewidth=2)
axes[0, 0].set_ylabel('Oil (bbl/month)', fontweight='bold')
axes[0, 0].set_title('Oil Production', fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# GOR
axes[0, 1].plot(multiphase_data.dates, historical_ratios['gor'], 'o-', label='Historical', linewidth=2)
axes[0, 1].plot(forecast_ratios['gor'].index, forecast_ratios['gor'].values, '-', label='Forecast', linewidth=2)
axes[0, 1].set_ylabel('GOR (SCF/STB)', fontweight='bold')
axes[0, 1].set_title('Gas-Oil Ratio', fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Water cut
axes[1, 0].plot(multiphase_data.dates, historical_ratios['water_cut'], 'o-', label='Historical', linewidth=2)
axes[1, 0].plot(forecast_ratios['water_cut'].index, forecast_ratios['water_cut'].values, '-', label='Forecast', linewidth=2)
axes[1, 0].set_ylabel('Water Cut (%)', fontweight='bold')
axes[1, 0].set_title('Water Cut', fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Gas production
axes[1, 1].plot(multiphase_data.dates, multiphase_data.gas.values, 'o-', label='Historical', linewidth=2)
axes[1, 1].plot(forecast['gas'].index, forecast['gas'].values, '-', label='Forecast', linewidth=2)
axes[1, 1].set_ylabel('Gas (MCF/month)', fontweight='bold')
axes[1, 1].set_title('Gas Production', fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
- ✓ Multi-phase data structures
- ✓ Simultaneous oil, gas, and water forecasting
- ✓ Maintaining physical relationships (GOR, water cut)
- ✓ Multi-phase visualization

### Next Steps
- Explore yield models for secondary phase forecasting
- Learn about multi-phase economic analysis
- Study consistency metrics for multi-phase forecasts